In [ ]:
!pip install langchain langchain_experimental

In [1]:
from langchain import SerpAPIWrapper
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain.utilities import ArxivAPIWrapper
from langchain_experimental.tools.python.tool import PythonAstREPLTool

from typing import Dict, Tuple
import os
import json

# from transformers import AutoModelForCausalLM, AutoTokenizer
# from transformers.generation import GenerationConfig

# 为了使用谷歌搜索（SERPAPI）， WolframAlpha，您需要自行申请它们的 API KEY，然后填入此处
os.environ['SERPAPI_API_KEY'] = ''
os.environ['WOLFRAM_ALPHA_APPID'] = ''

search = SerpAPIWrapper()
WolframAlpha = WolframAlphaAPIWrapper()
arxiv = ArxivAPIWrapper()
python=PythonAstREPLTool()

def tool_wrapper_for_qwen(tool):
    def tool_(query):
        query = json.loads(query)["query"]
        return tool.run(query)
    return tool_



# 以下是给千问看的工具描述：
TOOLS = [
    {
        'name_for_human':
            'google search',
        'name_for_model':
            'Search',
        'description_for_model':
            'useful for when you need to answer questions about current events.',
        'parameters': [{
            "name": "query",
            "type": "string",
            "description": "search query of google",
            'required': True
        }], 
        'tool_api': tool_wrapper_for_qwen(search)
    },
    {
        'name_for_human':
            'Wolfram Alpha',
        'name_for_model':
            'Math',
        'description_for_model':
            'Useful for when you need to answer questions about Math, Science, Technology, Culture, Society and Everyday Life.',
        'parameters': [{
            "name": "query",
            "type": "string",
            "description": "the problem to solved by Wolfram Alpha",
            'required': True
        }], 
        'tool_api': tool_wrapper_for_qwen(WolframAlpha)
    },  
    {
        'name_for_human':
            'arxiv',
        'name_for_model':
            'Arxiv',
        'description_for_model':
            'A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org.',
        'parameters': [{
            "name": "query",
            "type": "string",
            "description": "the document id of arxiv to search",
            'required': True
        }], 
        'tool_api': tool_wrapper_for_qwen(arxiv)
    },
    {
        'name_for_human':
            'python',
        'name_for_model':
            'python',
        'description_for_model':
            "A Python shell. Use this to execute python commands. When using this tool, sometimes output is abbreviated - Make sure it does not look abbreviated before using it in your answer. "
            "Don't add comments to your python code.",
        'parameters': [{
            "name": "query",
            "type": "string",
            "description": "a valid python command.",
            'required': True
        }],
        'tool_api': tool_wrapper_for_qwen(python)
    }

]

In [2]:
import boto3
from langchain.llms.bedrock import Bedrock
region="us-east-1"
boto3_bedrock = boto3.client(
    service_name="bedrock-runtime",
    region_name=region
)
llm_model_name = "anthropic.claude-v2"
parameters = {
    "max_tokens_to_sample": 8096,
    "stop_sequences": ["\nObservation"],
    "temperature":0.01,
    "top_p":0.85
}

model_id ="anthropic.claude-instant-v1" if llm_model_name == 'claude-instant' else "anthropic.claude-v2"
llm = Bedrock(model_id=model_id, client=boto3_bedrock, model_kwargs=parameters)

In [7]:
TOOL_DESC = """{name_for_model}: Call this tool to interact with the {name_for_human} API. What is the {name_for_human} API useful for? {description_for_model} Parameters: {parameters} Format the arguments as a JSON object."""

REACT_PROMPT = """Answer the following questions as best you can. You have access to the following tools:

{tool_descs}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action, should give the parameter name, use json fomate
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can be repeated zero or more times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

For example:
Question: Who is Leo DiCaprio's girlfriend?
Thought: To find out who Leo DiCaprio's current girlfriend is, I should do a web search.
Action: Search
Action Input: leo dicaprio girlfriend
Observation: Vittoria Ceretti's
Thought: I now know the final answer
Final Answer: Leo DiCaprio's girlfriend is Vittoria Ceretti's 

Begin!

Question: {query}"""

def build_planning_prompt(TOOLS, query):
    tool_descs = []
    tool_names = []
    for info in TOOLS:
        tool_descs.append(
            TOOL_DESC.format(
                name_for_model=info['name_for_model'],
                name_for_human=info['name_for_human'],
                description_for_model=info['description_for_model'],
                parameters=json.dumps(
                    info['parameters'], ensure_ascii=False),
            )
        )
        tool_names.append(info['name_for_model'])
    tool_descs = '\n\n'.join(tool_descs)
    tool_names = ','.join(tool_names)

    prompt = REACT_PROMPT.format(tool_descs=tool_descs, tool_names=tool_names, query=query)
    return prompt

prompt_1 = build_planning_prompt(TOOLS, query="加拿大2023年人口统计数字是多少？")
print(prompt_1)

Answer the following questions as best you can. You have access to the following tools:

Search: Call this tool to interact with the google search API. What is the google search API useful for? useful for when you need to answer questions about current events. Parameters: [{"name": "query", "type": "string", "description": "search query of google", "required": true}] Format the arguments as a JSON object.

Math: Call this tool to interact with the Wolfram Alpha API. What is the Wolfram Alpha API useful for? Useful for when you need to answer questions about Math, Science, Technology, Culture, Society and Everyday Life. Parameters: [{"name": "query", "type": "string", "description": "the problem to solved by Wolfram Alpha", "required": true}] Format the arguments as a JSON object.

Arxiv: Call this tool to interact with the arxiv API. What is the arxiv API useful for? A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Qu

In [8]:
response_1 = llm.predict(prompt_1)
response_1 = "\nThought" + response_1.split("Thought")[1]
print(response_1)

****************** bedrock prompt *****************
Answer the following questions as best you can. You have access to the following tools:

Search: Call this tool to interact with the google search API. What is the google search API useful for? useful for when you need to answer questions about current events. Parameters: [{"name": "query", "type": "string", "description": "search query of google", "required": true}] Format the arguments as a JSON object.

Math: Call this tool to interact with the Wolfram Alpha API. What is the Wolfram Alpha API useful for? Useful for when you need to answer questions about Math, Science, Technology, Culture, Society and Everyday Life. Parameters: [{"name": "query", "type": "string", "description": "the problem to solved by Wolfram Alpha", "required": true}] Format the arguments as a JSON object.

Arxiv: Call this tool to interact with the arxiv API. What is the arxiv API useful for? A wrapper around Arxiv.org Useful for when you need to answer questi

In [9]:
def parse_latest_plugin_call(text: str) -> Tuple[str, str]:
    i = text.rfind('\nAction:')
    j = text.rfind('\nAction Input:')
    k = text.rfind('\nObservation:')
    if 0 <= i < j:  # If the text has `Action` and `Action input`,
        if k < j:  # but does not contain `Observation`,
            # then it is likely that `Observation` is ommited by the LLM,
            # because the output text may have discarded the stop word.
            text = text.rstrip() + '\nObservation:'  # Add it back.
            k = text.rfind('\nObservation:')
    if 0 <= i < j < k:
        plugin_name = text[i + len('\nAction:'):j].strip()
        plugin_args = text[j + len('\nAction Input:'):k].strip()
        return plugin_name, plugin_args
    return '', ''

def use_api(tools, response):
    use_toolname, action_input = parse_latest_plugin_call(response)
    if use_toolname == "":
        return "no tool founds"

    used_tool_meta = list(filter(lambda x: x["name_for_model"] == use_toolname, tools))
    print(f'used_tool_meta: {used_tool_meta}')
    if len(used_tool_meta) == 0:
        return "no tool founds"

    print(f'action_input: {action_input}') 
    api_output = used_tool_meta[0]["tool_api"](action_input)
    return api_output

api_output = use_api(TOOLS, response_1)
print(api_output)



used_tool_meta: [{'name_for_human': 'google search', 'name_for_model': 'Search', 'description_for_model': 'useful for when you need to answer questions about current events.', 'parameters': [{'name': 'query', 'type': 'string', 'description': 'search query of google', 'required': True}], 'tool_api': <function tool_wrapper_for_qwen.<locals>.tool_ at 0x10bf893f0>}]
action_input: {"query": "加拿大2023年人口"}
根据加拿大统计局预测，加拿大人口今天（2023年6月16日）预计将超过4000万。 联邦统计局使用模型来实时估计加拿大的人口，该计数模型预计加拿大人口将在北美东部时间今天下午3点前达到4000万。 加拿大的人口增长率目前为2.7％。


In [10]:
prompt_2 = prompt_1 + response_1 + 'Observation: ' + api_output + "\nThought:"
stop = ["Observation:", "Observation:\n"]
response_2 = llm(prompt_2)
print(prompt_2)

print("*" * 250)
print(response_2)

****************** bedrock prompt *****************
Answer the following questions as best you can. You have access to the following tools:

Search: Call this tool to interact with the google search API. What is the google search API useful for? useful for when you need to answer questions about current events. Parameters: [{"name": "query", "type": "string", "description": "search query of google", "required": true}] Format the arguments as a JSON object.

Math: Call this tool to interact with the Wolfram Alpha API. What is the Wolfram Alpha API useful for? Useful for when you need to answer questions about Math, Science, Technology, Culture, Society and Everyday Life. Parameters: [{"name": "query", "type": "string", "description": "the problem to solved by Wolfram Alpha", "required": true}] Format the arguments as a JSON object.

Arxiv: Call this tool to interact with the arxiv API. What is the arxiv API useful for? A wrapper around Arxiv.org Useful for when you need to answer questi

In [13]:
api_output2 = use_api(TOOLS, response_2)
print(api_output2)
prompt_3 = prompt_2 + response_2 + '\nObservation: ' + api_output + "\nThought:"
stop = ["Observation:", "Observation:\n"]
response_3 = llm(prompt_3)
print(prompt_3)

print("*" * 250)
print(response_3)

used_tool_meta: [{'name_for_human': 'google search', 'name_for_model': 'Search', 'description_for_model': 'useful for when you need to answer questions about current events.', 'parameters': [{'name': 'query', 'type': 'string', 'description': 'search query of google', 'required': True}], 'tool_api': <function tool_wrapper_for_qwen.<locals>.tool_ at 0x10bf893f0>}]
action_input: {"query": "加拿大2023年人口"}
根据加拿大统计局预测，加拿大人口今天（2023年6月16日）预计将超过4000万。 联邦统计局使用模型来实时估计加拿大的人口，该计数模型预计加拿大人口将在北美东部时间今天下午3点前达到4000万。 加拿大的人口增长率目前为2.7％。
****************** bedrock prompt *****************
Answer the following questions as best you can. You have access to the following tools:

Search: Call this tool to interact with the google search API. What is the google search API useful for? useful for when you need to answer questions about current events. Parameters: [{"name": "query", "type": "string", "description": "search query of google", "required": true}] Format the arguments as a JSON object.

Math: Call this to